In [ ]:
%pip install pyswarms
%pip install tensorflow

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Sequential
from pyswarms.single.global_best import GlobalBestPSO
import numpy as np

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize and reshape the data
x_train = x_train / 255.0
x_test = x_test / 255.0
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

# Define the optimization function
def cnn_loss(p):
    # Set the hyperparameters
    filters = int(p[0])
    kernel_size = (int(p[1]), int(p[2]))
    strides = (int(p[3]), int(p[4]))
    padding = 'same' if p[5] <=0.5 else 'valid'
    activation = 'relu' if p[6] <= 0.5 else 'sigmoid'
    pool_size = (int(p[7]), int(p[8]))
    pool_strides = (int(p[9]), int(p[10]))
    pool_padding = 'same' if p[11] <= 0.5 else 'valid'
    units = int(p[12])
    dense_activation = 'relu' if p[13] <= 0.5 else 'sigmoid'

    # Define the model
    model = Sequential()

    # Add the Conv2D layer
    model.add(Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding=padding, activation=activation, input_shape=(28, 28, 1)))

    # Add the Pooling2D layer
    model.add(MaxPooling2D(pool_size=pool_size, strides=pool_strides, padding=pool_padding))

    # Flatten the output
    model.add(Flatten())

    # Add the Dense layer
    model.add(Dense(units=units, activation=dense_activation))

    # Add the Output layer
    model.add(tf.keras.layers.Dense(units=10, activation='softmax'))

    # Compile the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Fit the model
    model.fit(x_train, y_train, epochs=1)

    # Evaluate the model
    loss, accuracy = model.evaluate(x_test, y_test, verbose=0)

    # Return the negative loss as the optimization function minimizes the cost
    return -loss

def optimization_function(p):
  lossVector = np.apply_along_axis(cnn_loss, axis=1, arr=p)

  return lossVector

#cnn_loss([128, 3, 3, 3, 3, 0, 0, 4, 4, 3, 3, 0, 64, 0])
#optimization_func([[128, 3, 3, 3, 3, 0, 0, 4, 4, 3, 3, 0, 64, 0], [128, 3, 3, 3, 3, 0, 0, 4, 4, 3, 3, 0, 64, 0]])

# Set the search space
search_space = [
    (8, 256),  # filters
    (3, 7),    # kernel_size_x
    (3, 7),    # kernel_size_y
    (1, 3),    # strides_x
    (1, 3),    # strides_y
    (0, 1),    # padding
    (0, 1),    # activation
    (2, 4),    # pool_size_x
    (2, 4),    # pool_size_y
    (1, 3),    # pool_strides_x
    (1, 3),    # pool_strides_y
    (0, 1),    # pool_padding
    (32, 512), # units
    (0, 1)     # dense_activation
]

# Set the optimization options
options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}

# Set the bounds
lb = [a for a, _ in search_space]
ub = [b for _, b in search_space]
bounds = (lb, ub)
dimensions = len(lb)

# Initialize the optimizer
optimizer = GlobalBestPSO(n_particles=2, dimensions=dimensions, options=options, bounds=bounds)

# Run the optimization
cost, pos = optimizer.optimize(optimization_function, iters=1)

# Print the results
print("Best loss: ", -cost)
print("Best hyperparameters: ", pos)
print("")
print("Best conv layer filter number: " + str(int(pos[0])) )
print("Best conv layer kernel size: " + str((int(pos[1]), int(pos[2]))) )
print("Best conv layer strides: " + str((int(pos[3]), int(pos[4]))) )
print("Best conv layer padding: " + 'same' if pos[5] <=0.5 else "Best conv layer padding: " + 'valid' )
print("Best conv layer activation function: " + 'relu' if pos[6] <= 0.5 else "Best conv layer activation function: " + 'sigmoid' )
print("Best pool layer size: " + str((int(pos[7]), int(pos[8]))) )
print("Best pool layer strides: " + str((int(pos[9]), int(pos[10]))) )
print("Best pool layer padding: " + 'same' if pos[11] <= 0.5 else "Best pool layer padding: " + 'valid' )
print("Best fully connected layer (dense) units: " + str(int(pos[12])) )
print("Best fully connected layer (dense) function: " 'relu' if pos[13] <= 0.5 else "Best fully connected layer (dense) function: " + 'sigmoid' )


2023-01-02 05:56:14,286 - pyswarms.single.global_best - INFO - Optimize for 1 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best:   0%|          |0/1

1875/1875 [==============================] - 932s 496ms/step - loss: 2.7094 - accuracy: 0.1106


pyswarms.single.global_best: 100%|██████████|1/1, best_cost=-2.3
2023-01-02 06:15:31,544 - pyswarms.single.global_best - INFO - Optimization finished | best cost: -2.301082134246826, best pos: [1.31265690e+02 6.72042222e+00 6.61839278e+00 1.60892072e+00
 2.74324448e+00 8.98791694e-02 5.46565947e-01 2.13648509e+00
 3.48889883e+00 1.40649130e+00 2.31246226e+00 6.12850830e-01
 9.72564803e+01 9.97871671e-02]


Best loss:  2.301082134246826
Best hyperparameters:  [1.31265690e+02 6.72042222e+00 6.61839278e+00 1.60892072e+00
 2.74324448e+00 8.98791694e-02 5.46565947e-01 2.13648509e+00
 3.48889883e+00 1.40649130e+00 2.31246226e+00 6.12850830e-01
 9.72564803e+01 9.97871671e-02]

Best conv layer filter number: 131
Best conv layer kernel size: (6, 6)
Best conv layer strides: (1, 2)
Best conv layer padding: same
Best conv layer activation function: sigmoid
Best pool layer size: (2, 3)
Best pool layer strides: (1, 2)
Best pool layer padding: valid
Best fully connected layer (dense) units: 97
Best fully connected layer (dense) function: relu
